In [15]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv("./train.csv")


In [4]:
data = np.array(data) # Convert data to array with numpy
m, n = data.shape # Dimensions of the data. m = number of entries, n = number of pixels
np.random.shuffle(data) # Shuffle data before splitting into training and dev sets

data_dev = data[0:1000].T # uses first 1000 examples for dev sets and transposes it
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T # uses the rest as training data
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_, m_train = X_train.shape

In [5]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forw_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [6]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forw_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [7]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 1000)

Iteration:  0
[6 6 1 ... 0 6 5] [4 9 6 ... 7 6 3]
0.11978048780487804
Iteration:  10
[6 0 1 ... 1 2 2] [4 9 6 ... 7 6 3]
0.18817073170731707
Iteration:  20
[6 4 1 ... 1 2 2] [4 9 6 ... 7 6 3]
0.24985365853658537
Iteration:  30
[6 4 1 ... 7 2 1] [4 9 6 ... 7 6 3]
0.3230487804878049
Iteration:  40
[6 9 1 ... 7 2 1] [4 9 6 ... 7 6 3]
0.38709756097560977
Iteration:  50
[6 9 1 ... 7 2 1] [4 9 6 ... 7 6 3]
0.45234146341463416
Iteration:  60
[6 9 1 ... 7 2 1] [4 9 6 ... 7 6 3]
0.5078536585365854
Iteration:  70
[4 7 0 ... 7 6 1] [4 9 6 ... 7 6 3]
0.5571707317073171
Iteration:  80
[4 9 0 ... 7 6 1] [4 9 6 ... 7 6 3]
0.5859756097560975
Iteration:  90
[4 9 0 ... 7 6 1] [4 9 6 ... 7 6 3]
0.609390243902439
Iteration:  100
[4 9 0 ... 7 6 1] [4 9 6 ... 7 6 3]
0.6274634146341463
Iteration:  110
[4 9 0 ... 7 6 1] [4 9 6 ... 7 6 3]
0.6454878048780488
Iteration:  120
[4 9 0 ... 7 6 2] [4 9 6 ... 7 6 3]
0.6605609756097561
Iteration:  130
[4 9 0 ... 7 6 2] [4 9 6 ... 7 6 3]
0.6742439024390244
Iteration:  1

In [8]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forw_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [9]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[4 0 1 6 7 9 3 1 7 8 3 4 6 1 0 9 2 7 3 6 4 3 8 9 5 5 0 4 3 2 7 7 1 3 6 6 0
 7 4 3 7 4 1 2 1 9 3 7 1 7 3 4 1 7 8 9 6 1 5 5 5 0 7 1 4 4 1 6 8 0 1 7 5 6
 3 2 3 3 1 8 0 3 7 8 5 1 9 5 8 9 7 7 9 5 7 6 9 5 1 3 0 0 9 4 9 3 6 4 5 7 7
 4 7 5 9 1 3 9 5 5 3 5 8 1 5 0 4 4 0 7 8 2 8 8 7 7 6 4 0 1 1 0 6 3 4 8 7 9
 4 6 1 3 9 0 3 5 1 2 4 5 7 8 7 9 4 6 2 6 7 4 5 9 7 1 9 4 8 5 5 4 2 5 7 5 9
 0 6 5 5 4 7 6 0 6 8 8 0 1 3 8 0 2 1 6 6 6 5 4 6 0 2 6 5 6 7 8 4 9 5 8 6 7
 6 9 5 9 8 8 1 0 0 8 4 1 2 9 9 9 1 9 0 5 1 9 8 2 4 1 6 4 3 5 4 0 2 9 0 7 4
 6 9 3 5 2 2 6 1 1 9 7 2 0 4 1 9 7 4 4 3 7 0 7 6 7 6 4 8 3 3 0 7 5 4 1 2 5
 1 8 9 4 0 0 8 7 9 0 9 7 9 2 8 2 6 8 8 8 7 7 6 6 6 4 9 1 9 9 1 9 9 7 8 1 3
 9 6 8 4 0 5 1 0 3 6 8 2 4 1 1 2 3 3 7 8 6 2 6 1 2 3 5 6 0 2 3 6 5 3 3 1 1
 0 8 5 7 5 4 5 1 2 2 1 3 6 5 1 3 5 0 7 0 9 1 2 8 9 0 3 0 4 5 3 6 7 5 3 7 3
 9 0 1 2 3 4 7 7 9 3 7 4 0 4 3 5 2 7 6 2 8 9 4 3 8 2 2 7 8 9 8 9 4 5 1 5 5
 7 0 8 0 1 0 8 0 7 5 5 2 7 2 5 1 8 9 4 4 8 3 9 3 5 2 7 3 7 5 3 5 6 1 8 8 8
 6 7 3 6 7 5 1 3 1 0 3 9 

np.float64(0.87)